In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import logging
import yaml
import sys
import requests
import json
import yfinance as yf
from typing import List, Set, Tuple
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

from backtesting.Backtest import Backtest

# Set up logging with more detailed format
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    stream=sys.stdout,  # Ensure output goes to notebook
    level=logging.INFO
)
logger = logging.getLogger(__name__)

# Test logger to ensure it prints to the output
logger.info("Logger is set up and ready to capture info statements.")

/Users/calvinseamons/Nexus/quant-trading-ml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-02-03 12:49:36,437 - __main__ - INFO - Logger is set up and ready to capture info statements.


In [7]:
def setup_directories():
    """Create necessary directories for the backtest system."""
    try:
        # Define base directories
        __file__ = os.path.abspath('main.ipynb')
        base_dir = os.path.abspath(os.path.dirname(__file__))
        data_dir = os.path.join(base_dir, 'data')
        db_dir = os.path.join(data_dir, 'db')
        cache_dir = os.path.join(data_dir, 'cache')
        
        logger.debug(f"Setting up directories:")
        logger.debug(f"Base dir: {base_dir}")
        logger.debug(f"Data dir: {data_dir}")
        logger.debug(f"DB dir: {db_dir}")
        logger.debug(f"Cache dir: {cache_dir}")
        
        # Create directories
        Path(data_dir).mkdir(exist_ok=True)
        Path(db_dir).mkdir(exist_ok=True)
        Path(cache_dir).mkdir(exist_ok=True)
        
        # Verify directories were created
        for dir_path in [data_dir, db_dir, cache_dir]:
            if not os.path.exists(dir_path):
                raise RuntimeError(f"Failed to create directory: {dir_path}")
            else:
                logger.debug(f"Verified directory exists: {dir_path}")
        
        # Create default config if it doesn't exist
        config_path = os.path.join(base_dir, 'config.yaml')
        if not os.path.exists(config_path):
            logger.debug("Creating default config.yaml")
            default_config = {
                'cache': {
                    'max_memory_cache_size': 1000,
                    'cache_expiry_days': 1,
                    'update_frequency': '1d',
                    'compression_type': 'parquet'
                },
                'download': {
                    'max_retries': 3,
                    'retry_delay': 5,
                    'batch_size': 100,
                    'timeout': 30
                },
                'validation': {
                    'min_data_points': 50,
                    'max_missing_pct': 0.1,
                    'price_threshold': 0.01
                }
            }
            with open(config_path, 'w') as f:
                yaml.dump(default_config, f)
            logger.debug("Created config.yaml successfully")
        else:
            logger.debug("config.yaml already exists")
        
        return {
            'base_dir': base_dir,
            'data_dir': data_dir,
            'db_dir': db_dir,
            'cache_dir': cache_dir,
            'config_path': config_path
        }
    except Exception as e:
        logger.error(f"Error in setup_directories: {str(e)}")
        raise

In [8]:
start_date = '2022-01-01'
end_date = '2024-01-01'

symbol = 'AAPL'

hist = yf.download(symbol, 
                    start=start_date, 
                    end=end_date, 
                    progress=False)
        # Check if we have data
print(len(hist))

501


In [9]:
def verify_data_availability(symbol: str, start_date: str, end_date: str) -> bool:
    """
    Verifies if a symbol has any data available in the specified date range.
    """
    try:
        hist = yf.download(symbol, 
                          start=start_date, 
                          end=end_date, 
                          progress=False)
        
        return not hist.empty
        
    except Exception as e:
        logger.debug(f"Error verifying {symbol}: {str(e)}")
        return False

def get_combined_symbols(num_sp500: int, num_sec: int, start_date: str, end_date: str, logger) -> Tuple[List[str], List[str]]:
    """
    Gets two lists of symbols that have data: S&P 500 and SEC stocks.
    """
    # Get S&P 500 symbols
    try:
        url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
        tables = pd.read_html(url)
        sp500_symbols = tables[0]['Symbol'].tolist()
        sp500_symbols = [symbol.replace(".", "-") for symbol in sp500_symbols]
        logger.info(f"Retrieved {len(sp500_symbols)} S&P 500 symbols")
    except Exception as e:
        logger.error(f"Error retrieving S&P 500 symbols: {str(e)}")
        sp500_symbols = []

    # Get SEC symbols
    try:
        sec_url = "https://www.sec.gov/files/company_tickers.json"
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(sec_url, headers=headers)
        data = response.json()
        sec_symbols = {company['ticker'] for company in data.values()}
        logger.info(f"Retrieved {len(sec_symbols)} SEC symbols")
    except Exception as e:
        logger.error(f"Error retrieving SEC symbols: {str(e)}")
        sec_symbols = set()

    sp500_verified = []
    sec_verified = []
    
    # Verify S&P 500 symbols
    logger.info("Verifying S&P 500 symbols...")
    for symbol in sp500_symbols:
        if len(sp500_verified) >= num_sp500:
            break
        if verify_data_availability(symbol, start_date, end_date):
            sp500_verified.append(symbol)

    # Verify SEC symbols (excluding S&P 500)
    logger.info("Verifying SEC symbols...")
    sp500_set = set(sp500_symbols)
    additional_symbols = [sym for sym in sec_symbols if sym not in sp500_set]
    
    for symbol in additional_symbols:
        if len(sec_verified) >= num_sec:
            break
        if verify_data_availability(symbol, start_date, end_date):
            sec_verified.append(symbol)

    logger.info(f"Found {len(sp500_verified)} S&P 500 symbols and {len(sec_verified)} SEC symbols with data")
    return sp500_verified, sec_verified

In [11]:
START_DATE = '2023-01-01'
END_DATE = '2024-01-01'

# Get symbols
sp500_symbols, sec_symbols = get_combined_symbols(500, 100, START_DATE, END_DATE, logger)
print(f"Retrieved {len(sp500_symbols)} verified symbols")
print(f"Retrieved {len(sec_symbols)} verified symbols")

2025-02-03 12:53:53,004 - __main__ - INFO - Retrieved 503 S&P 500 symbols
2025-02-03 12:53:53,383 - __main__ - INFO - Retrieved 9761 SEC symbols
2025-02-03 12:53:53,384 - __main__ - INFO - Verifying S&P 500 symbols...
2025-02-03 12:54:02,911 - yfinance - ERROR - 
1 Failed download:
2025-02-03 12:54:02,912 - yfinance - ERROR - ['BKNG']: ConnectionError(ProtocolError('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))
2025-02-03 12:54:24,924 - yfinance - ERROR - 
1 Failed download:
2025-02-03 12:54:24,925 - yfinance - ERROR - ['GEV']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2024-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1704085200")')
2025-02-03 12:54:55,031 - yfinance - ERROR - 
1 Failed download:
2025-02-03 12:54:55,031 - yfinance - ERROR - ['SW']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2024-01-01) (Yahoo 

In [15]:
verified_symbols = sp500_symbols + sec_symbols

print(len(verified_symbols))

# Use with your backtest
bt = Backtest(config_path='config/config.yaml', start_date=START_DATE, end_date=END_DATE)
data = bt.fetch_historical_data(verified_symbols)

599
2025-02-03 12:57:30,048 - backtesting.Backtest - INFO - Starting fetch_historical_data for 599 symbols
2025-02-03 12:57:30,049 - backtesting.Backtest - INFO - Date range: 2023-01-01 00:00:00 to 2024-01-01 00:00:00
2025-02-03 12:57:30,049 - backtesting.Backtest - INFO - Creating new BacktestDataManager instance
Loaded configuration from config/config.yaml
2025-02-03 12:57:30,060 - backtesting.Backtest - INFO - Calling data_manager.get_data

=== Starting get_data ===
Input symbols: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A', 'APD', 'ABNB', 'AKAM', 'ALB', 'ARE', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APA', 'APO', 'AAPL', 'AMAT', 'APTV', 'ACGL', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'AXON', 'BKR', 'BALL', 'BAC', 'BAX', 'BDX', 'BRK-B', 'BBY', 'TECH', 'BIIB', 'BLK', 'BX', 'BK', 'BA', 'BWA', 'BSX', 'BM

In [16]:
def analyze_symbol_data(data_dict):
    """
    Analyze a dictionary containing symbols as keys and DataFrames as values.
    
    Args:
        data_dict (dict): A dictionary where keys are symbol strings and values are DataFrames.
    
    Returns:
        None
    """
    if not isinstance(data_dict, dict):
        print("The input is not a dictionary.")
        return
    
    print(f"Total symbols: {len(data_dict)}\n")
    
    for symbol, df in data_dict.items():
        print(f"Symbol: {symbol}")
        if isinstance(df, pd.DataFrame):
            print(f"  Shape: {df.shape}")
            print(f"  Columns: {list(df.columns)}")
            print(f"  Data Types:\n{df.dtypes}")
            print(f"  Memory Usage: {df.memory_usage(deep=True).sum()} bytes")
            print(f"  First 5 rows:\n{df.head()}\n")
        else:
            print(f"  Warning: Value is not a DataFrame (type: {type(df)})\n")
            
# Example usage
# Assuming `symbol_data` is your dictionary
analyze_symbol_data(data)


Total symbols: 553

Symbol: ABBV
  Shape: (250, 7)
  Columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
  Data Types:
Open            float64
High            float64
Low             float64
Close           float64
Volume            int64
Dividends       float64
Stock Splits    float64
dtype: object
  Memory Usage: 16000 bytes
  First 5 rows:
                                 Open        High         Low       Close  \
Date                                                                        
2023-01-03 00:00:00-05:00  148.851800  149.752050  147.721912  149.164139   
2023-01-04 00:00:00-05:00  149.182476  150.909470  148.364913  150.367493   
2023-01-05 00:00:00-05:00  149.816324  151.102388  149.127365  150.183777   
2023-01-06 00:00:00-05:00  151.423874  154.427736  150.762473  152.994705   
2023-01-09 00:00:00-05:00  152.397635  152.746712  147.786216  148.502731   

                            Volume  Dividends  Stock Splits  
Date                    

In [17]:
# Check if VLTO is in the data
if 'VLTO' in data:
    print("VLTO is still in the data!")
    print(f"VLTO data length: {len(data['VLTO'])}")
else:
    print("VLTO was successfully filtered out")

VLTO was successfully filtered out


In [ ]:
from models.LSTMManager import LSTMManager

# test_symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META']
# historical_data = bt.fetch_historical_data(test_symbols)

model = LSTMManager()
model.train(data)

2025-02-03 12:58:11,966 - features.FeatureEngineer - INFO - FeatureEngineering initialized with multi-index support
[DEBUG] Initializing DirectionalLSTM with config input_size=78
2025-02-03 12:58:11,966 - root - INFO - LSTMManager initialized with multi-index support
2025-02-03 12:58:11,966 - root - INFO - Starting training with multi-index support...

[DEBUG] Initial historical data symbols: ['ABBV', 'AOS', 'MMM', 'ABT', 'ACN', 'ADBE', 'A', 'AMD', 'AES', 'AFL', 'APD', 'ABNB', 'AKAM', 'ALGN', 'ALB', 'ARE', 'ALLE', 'LNT', 'GOOG', 'ALL', 'AMZN', 'AMCR', 'GOOGL', 'MO', 'AEE', 'AMT', 'AWK', 'AXP', 'AEP', 'AIG', 'AMP', 'AMGN', 'ANSS', 'AME', 'ADI', 'APH', 'AON', 'APO', 'APA', 'AMAT', 'AAPL', 'APTV', 'ACGL', 'ANET', 'ADM', 'AIZ', 'AJG', 'T', 'ATO', 'AZO', 'ADP', 'AVB', 'AXON', 'ADSK', 'AVY', 'BKR', 'BALL', 'BAX', 'BRK-B', 'BDX', 'BBY', 'TECH', 'BAC', 'BIIB', 'BX', 'BLK', 'BWA', 'BSX', 'BA', 'BK', 'AVGO', 'BR', 'BMY', 'BRO', 'BLDR', 'BF-B', 'BXP', 'CDNS', 'CHRW', 'BG', 'CZR', 'CPT', 'KMX', 'C

/Users/calvinseamons/Nexus/quant-trading-ml/src/features/Features.py:34: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features['log_volume'] = np.log(data['Volume'].clip(lower=epsilon))
/Users/calvinseamons/Nexus/quant-trading-ml/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/calvinseamons/Nexus/quant-trading-ml/src/features/Features.py:34: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all

2025-02-03 12:58:15,121 - features.FeatureEngineer - INFO - Generating volatility features
2025-02-03 12:58:15,126 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:58:15,142 - features.FeatureEngineer - INFO - Processing features for symbol BLK
2025-02-03 12:58:15,143 - features.FeatureEngineer - INFO - Generating price features
2025-02-03 12:58:15,148 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:58:15,152 - features.FeatureEngineer - INFO - Generating momentum features
2025-02-03 12:58:15,161 - features.FeatureEngineer - INFO - Generating volatility features
2025-02-03 12:58:15,166 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:58:15,180 - features.FeatureEngineer - INFO - Processing features for symbol BMY
2025-02-03 12:58:15,181 - features.FeatureEngineer - INFO - Generating price features
2025-02-03 12:58:15,187 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:58:

/Users/calvinseamons/Nexus/quant-trading-ml/src/features/Features.py:34: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features['log_volume'] = np.log(data['Volume'].clip(lower=epsilon))
/Users/calvinseamons/Nexus/quant-trading-ml/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/Users/calvinseamons/Nexus/quant-trading-ml/src/features/Features.py:34: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all

2025-02-03 12:58:15,431 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:58:15,444 - features.FeatureEngineer - INFO - Processing features for symbol BTO
2025-02-03 12:58:15,446 - features.FeatureEngineer - INFO - Generating price features
2025-02-03 12:58:15,451 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:58:15,455 - features.FeatureEngineer - INFO - Generating momentum features
2025-02-03 12:58:15,463 - features.FeatureEngineer - INFO - Generating volatility features
2025-02-03 12:58:15,468 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:58:15,482 - features.FeatureEngineer - INFO - Processing features for symbol BWA
2025-02-03 12:58:15,483 - features.FeatureEngineer - INFO - Generating price features
2025-02-03 12:58:15,489 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:58:15,493 - features.FeatureEngineer - INFO - Generating momentum features
2025-02-03 12:58:15

...

2025-02-03 12:59:20,819 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:59:20,822 - features.FeatureEngineer - INFO - Generating momentum features
2025-02-03 12:59:20,830 - features.FeatureEngineer - INFO - Generating volatility features
2025-02-03 12:59:20,834 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:59:20,867 - features.FeatureEngineer - INFO - Processing features for symbol TPL
2025-02-03 12:59:20,870 - features.FeatureEngineer - INFO - Generating price features
2025-02-03 12:59:20,875 - features.FeatureEngineer - INFO - Generating returns features
2025-02-03 12:59:20,879 - features.FeatureEngineer - INFO - Generating momentum features
2025-02-03 12:59:20,886 - features.FeatureEngineer - INFO - Generating volatility features
2025-02-03 12:59:20,890 - features.FeatureEngineer - INFO - Generating volume features
2025-02-03 12:59:20,902 - features.FeatureEngineer - INFO - Processing features for symbol TPR
2025-02-03 12:

/Users/calvinseamons/Nexus/quant-trading-ml/src/features/Features.py:34: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  features['log_volume'] = np.log(data['Volume'].clip(lower=epsilon))
/Users/calvinseamons/Nexus/quant-trading-ml/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


- Scaled train shape: (110948, 78)
- Scaled val shape: (25938, 78)
- Sample scaled values range: [-1539316278886397.750, 274877906943998.750]

[DEBUG] Creating sequences...
- Train sequences shape: (94440, 20, 78)
- Train labels shape: (94440,)
- Val sequences shape: (11662, 20, 78)
- Val labels shape: (11662,)
- Label distribution - Train: [54393 40047]
- Label distribution - Val: [5390 6272]

[DEBUG] Sequence validation passed

[DEBUG] Computed class weights: {0: np.float64(0.8681264133252441), 1: np.float64(1.1791145404150123)}

[DEBUG] DataLoader details:
- Train batches: 2952
- Val batches: 365
- Batch size: 32

[DEBUG] Starting model training...
[DEBUG] Initializing model layers...
[DEBUG] Initializing layers with actual input_size=78


/Users/calvinseamons/Nexus/quant-trading-ml/.venv/lib/python3.10/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


[DEBUG] Model layers initialized successfully
[DEBUG] Configuring optimizer and scheduler...
[DEBUG] Optimizer and scheduler configured successfully
[DEBUG] Starting training loop...
Epoch 1/100:
Train Loss: 0.6834, Train Acc: 55.35%
Val Loss: 0.7058, Val Acc: 48.07%
Epoch 2/100:
Train Loss: 0.6674, Train Acc: 58.61%
Val Loss: 0.8086, Val Acc: 36.52%


In [ ]:
model.optimize_hyperparameters(historical_data)